# 5.6.1 Affine 계층

np.dot으로 행렬의 곱셈을 표현했던 것을 기억해보자.

In [2]:
import numpy as np

X = np.random.rand(2)    # input
W = np.random.rand(2, 3) # weight
B = np.random.rand(3)    # bias

print(X.shape)
print(W.shape)
print(B.shape)

np.dot(X, W) + B

(2,)
(2, 3)
(3,)


array([0.95327695, 0.80396679, 0.67617759])

행렬의 곱은 아래와 같은 특징이 있다.

* 행렬곱을 하기 위해서는 앞/뒤의 차원 수가 같아야 한다.
* 행렬의 곱을 기하학에서는 Affine 변환이라고 한다.

이 Affine 변환 역시 계산 그래프로 표현 가능하다.


<p align="center"><img src="imgs/5-24.png" width=600></p>

왜 이제와서 affine이 나오는가? 지금까지의 계산은 스칼라값이 흘렀는데, 이제 행렬이 흐르고 있기 때문이다. 

* 아래 예시에서는 편의상 backward에 x를 그대로 넣었다
* 하지만 실제 계산에서는, global loss로부터 계산된 배열이 들어간다.
* 그러니 값들은 그냥 참고용으로만 알아두자.

좋은 자료: 오차 역전파를 계산그래프로 설명하는 유튜브 영상
https://www.youtube.com/watch?v=1Q_etC_GHHk&t=606s

# 5.6.2 배치용 Affine 계층

In [4]:
X_dot_W = np.array([[0, 0, 0], [10, 10, 10]])
B = np.array([1, 2, 3])

print('X_dot_W: \n', X_dot_W)
print('B:', B)

X_dot_W: 
 [[ 0  0  0]
 [10 10 10]]
B: [1 2 3]


In [5]:
X_dot_W + B

array([[ 1,  2,  3],
       [11, 12, 13]])

위와 같은 현상이 일어나는 것은 numpy가 가지고 있는 broadcasting 기능 덕분이다. 이는 shape가 작은 행렬이 더 큰 행렬에 맞춰서 알아서 크기가 조절되는 것을 말한다. (확장)

In [6]:
dY = np.array([[1, 2, 3], [4, 5, 6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [8]:
dB = np.sum(dY, axis=0)
dB

array([5, 7, 9])

Affine 계층의 bias 계산되는 원리를 이해하면 된다.

In [9]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        # 가중치와 편향 매개변수의 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        # 텐서 대응
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)  # 입력 데이터 모양 변경(텐서 대응)
        return dx